In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import argparse

In [3]:
import torch
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torchvision.models as models
import os
import argparse
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
%pip install wandb -q
import wandb

Note: you may need to restart the kernel to use updated packages.


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 6, kernel_size =5)
        self.fc1 = nn.Linear(6*5*5, 120)
        self.fc2 = nn.Linear(120, 84 )
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x 

def convmodel():
    model = Net()
    return model

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best accuracy on test
device

'cuda'

In [ ]:
print('transform data')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [ ]:
validation_split = .25
random_seed = 1
shuffle_dataset = True

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

# find data indices for training and validation splits:
dataset_size = len(trainset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,num_workers=2,sampler=train_sampler)

validationloader = torch.utils.data.DataLoader(trainset, batch_size=128, num_workers=2,sampler=valid_sampler)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# Training
def train(epoch,net,criterion,optimizer):
    print('\nEpoch: %d' % epoch)
    net.train()

    running_loss = 0.0
    val_running_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)       
        optimizer.zero_grad() # zero the parameter gradients

        # forward + backward + optimize
        outputs = net(inputs).to(device)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    running_loss_history.append(running_loss)#for each epoch log training losses in losstrain.txt
    
    
    losstrain={'epoch': [epoch],'loss':[running_loss]}## store epoch and losses dictionary
    
    loss_df = pd.DataFrame.from_dict(losstrain)## create dataframe from this dictionary
    
    loss_df.to_csv(r'train_loss.txt', header=None, index=None, sep=' ', mode='a')## dataframe to txt file
    print('Finished Training')
    print('Training Loss: %d' % running_loss)
    wandb.log({"Epoch":epoch,"Training Loss": running_loss})

    with torch.no_grad(): # we do not need gradient for validation.
      for val_inputs, val_labels in validationloader:
        val_inputs = val_inputs.to(device)
        val_labels = val_labels.to(device)
        val_outputs = net(val_inputs)
        val_loss = criterion(val_outputs, val_labels)
        
        _, val_preds = torch.max(val_outputs, 1)
        val_running_loss += val_loss.item()
    val_running_loss_history.append(val_running_loss)
    wandb.log({"Epoch":epoch,"Training Loss": running_loss})

In [ ]:
def test(epoch,net,criterion):
  global best_acc
  net.eval()
  total = 0
  correct = 0
  test_loss = 0
  test_images = []
  print('\nEpoch: %d' % epoch)
  with torch.no_grad():
      for batch_idx, (inputs, targets) in enumerate(testloader):
          inputs, targets = inputs.to(device), targets.to(device)
          ## Giving input to the model
          outputs = net(inputs)
          loss = criterion(outputs, targets)
          ## Predicting on the input data
          _, predicted = torch.max(outputs.data, 1)
          ## Defining the total size of the target
          total += targets.size(0)
          ## Getting the total number of correctly classified labels
          test_loss += loss.item()
          correct += (predicted == targets).sum().item()
          ## Appending images to image list
          test_images.append(wandb.Image(inputs[0], caption="Pred: {} Truth: {}".format(predicted[0].item(), targets[0])))
          ## Getting the accuracy of the model
      model_accuracy = 100 * correct / total
      print('Testing Accuracy: %d ' % model_accuracy)
      print('\n')
      ## Checking whether the accuracy is better than the best accuracy
      if model_accuracy>best_acc:
      ## Saving the new value of best_accuracy
        best_acc = model_accuracy
        ## Saving the model
        # Save checkpoint for the model which yields best accuracy
        PATH = './cifar_net.pth'
        torch.save(net.state_dict(), PATH)
          # Write test set losses to losstest.txt for each epoch
      df={'epoch': [epoch],'loss':[test_loss]}
      data = pd.DataFrame.from_dict(df)
      data.to_csv(r'losstest.txt', header=None, index=None, sep=' ', mode='a')
      #Write test set accuracies to acctest.txt for each epoch
      testaccuracy={'epoch': [epoch],'test_accuracy':[model_accuracy]}
      ## Making a dataframe from this dictionary
      accuracy_df = pd.DataFrame.from_dict(testaccuracy)
      ## Making a txt file from this dataframe
      accuracy_df.to_csv(r'test_accuracy.txt', header=None, index=None, sep=' ', mode='a')
      wandb.log({"Epoch":epoch,"Testing_images": test_images,"Test Accuracy": 100. * correct / len(testloader.dataset),"Test Loss": test_loss})   
  # Write test set losses to losstest.txt for each epoch

In [ ]:
def training_testing(net,criterion,optimizer):
  n_epochs_stop = 20
  epochs_no_improve = 0
  for epoch in range(0,500):
    print("Model Training")
    train(epoch,net,criterion,optimizer)
    if val_running_loss_history[epoch]>val_running_loss_history[epoch-1]:
      if(epochs_no_improve<n_epochs_stop):
        epochs_no_improve+=1
      else:
        break
    else:
      epochs_no_improve=0
    print("\n Model Testing")
    test(epoch,net,criterion)

In [ ]:
def best_param_prediction(net):
  ## Loading the best model parameters 
  PATH ='./cifar_net.pth'
  net.load_state_dict(torch.load(PATH))
  # prepare to count predictions for each class
  correct_pred = {classname: 0 for classname in classes}
  total_pred = {classname: 0 for classname in classes}

  # again no gradients needed
  with torch.no_grad():
      for batch_id, (images, labels) in enumerate(testloader):
          images, labels = images.to(device), labels.to(device)
          outputs = net(images)
          _, predictions = torch.max(outputs, 1)
          # collect the correct predictions for each class
          for label, prediction in zip(labels, predictions):
              if label == prediction:   
                  correct_pred[classes[label]] += 1
              total_pred[classes[label]] += 1


  # print accuracy for each class
  for classname, correct_count in correct_pred.items():
      accuracy = 100 * float(correct_count) / total_pred[classname]
      print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                    accuracy))

In [ ]:
def plot_confusion_matrix(net):
  total_correct = 0
  total_images = 0
  confusion_matrix = np.zeros([10,10], int)
  with torch.no_grad():
      for batch_id, (images, labels) in enumerate(testloader):
          images, labels = images.to(device), labels.to(device)
          outputs = net(images)
          _, predicted = torch.max(outputs.data, 1)
          total_images += labels.size(0)
          total_correct += (predicted == labels).sum().item()
          for i, l in enumerate(labels):
              confusion_matrix[l.item(), predicted[i].item()] += 1 

  model_accuracy = total_correct / total_images * 100
  print('Model accuracy on {0} test images: {1:.2f}%'.format(total_images, model_accuracy))
  fig, ax = plt.subplots(1,1,figsize=(8,6))
  sns.heatmap(confusion_matrix,annot = True,fmt='d')
  plt.ylabel('Actual Category')
  plt.yticks(range(10), classes)
  plt.xlabel('Predicted Category')
  plt.xticks(range(10), classes)
  plt.show()

In [ ]:
# ! rm -rf ./losstest.txt
# ! rm -rf ./train_loss.txt
# ! rm -rf ./test_accuracy.txt
# ! rm -rf ./cifar_net.pth

Model 1

In [ ]:
run=wandb.init(project="CIFAR_20BM6JP15_FADML_Final",reinit=True)
wandb.watch_called = False
net1 = convmodel()
wandb.watch(net1,log="all")
net1 = net1.to(device)
running_loss_history = []
val_running_loss_history = []
### Defining a Loss function and optimizer for configuration 1
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net1.parameters(), lr=0.001, momentum=0.9)
training_testing(net1,criterion,optimizer)

In [ ]:
best_param_prediction(net1)

In [ ]:
plot_confusion_matrix(net1)

Model 2

In [ ]:
run=wandb.init(project="CIFAR_20BM6JP15_FADML_Final",reinit=True)
wandb.watch_called = False
net2 = convmodel()
wandb.watch(net2,log="all")
net2 = net2.to(device)
running_loss_history = []
val_running_loss_history = []
### Defining a Loss function and optimizer for configuration 1
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net2.parameters(), lr=0.01)
training_testing(net2,criterion,optimizer)

In [ ]:
best_param_prediction(net2)

In [ ]:
plot_confusion_matrix(net2)

Model 3

In [ ]:
run=wandb.init(project="CIFAR_20BM6JP15_FADML_Final",reinit=True)
wandb.watch_called = False
net3 = convmodel()
wandb.watch(net1,log="all")
net3 = net3.to(device)
running_loss_history = []
val_running_loss_history = []
### Defining a Loss function and optimizer for configuration 1
criterion = nn.MSELoss()
optimizer = optim.SGD(net3.parameters(), lr=0.001, momentum=0.9)
training_testing(net3,criterion,optimizer)

In [ ]:
best_param_prediction(net3)

In [ ]:
plot_confusion_matrix(net3)

Model 4

In [ ]:
run=wandb.init(project="CIFAR_20BM6JP15_FADML_Final",reinit=True)
wandb.watch_called = False
net4 = convmodel()
wandb.watch(net1,log="all")
net4 = net4.to(device)
running_loss_history = []
val_running_loss_history = []
### Defining a Loss function and optimizer for configuration 1
criterion = nn.MSELoss()
optimizer = optim.Adam(net2.parameters(), lr=0.01)
training_testing(net4,criterion,optimizer)

In [ ]:
best_param_prediction(net4)

In [ ]:
plot_confusion_matrix(net4)

Resnet Model

In [ ]:
rn34 = models.resnet34(pretrained=True)
rn34

In [ ]:
for param in rn34.parameters():
  param.requires_grad = False
rn34.fc = nn.Sequential(nn.Linear(512,256),
                         nn.ReLU(),
                         nn.Linear(256,128),
                         nn.ReLU(),
                         nn.Linear(128,10),
                         nn.Softmax(dim =1))

In [ ]:
run=wandb.init(project="CIFAR_20BM6JP15_FADML_Final",reinit=True)
wandb.watch_called = False
rn34 = rn34.to(device)
wandb.watch(rn34,log="all")
running_loss_history = []
val_running_loss_history = []

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rn34.parameters(), lr=0.001)
training_testing(rn34,criterion,optimizer)

In [ ]:
best_param_prediction(rn34)

In [ ]:
plot_confusion_matrix(rn34)